In [1]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langgraph.graph import StateGraph, END

In [2]:
df = pd.read_csv("..\data\processed\Articulos_LLM.csv")

In [3]:
docs = [
    Document(
        page_content=row["contenido"],
        metadata={
            "titulo": row["titulo"],
            "url": row["url"],
            "precio": row.get("Precios", ""),
            "fecha": row.get("fechas", ""),
            "contexto_fecha":row.get("fechas_contexto",""),
            "edad": row.get("edad", ""),
        }
    )
    for _, row in df.iterrows()
]


In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)

# Crear los Document chunked
chunked_docs = []

for _, row in df.iterrows():
    texto = row["contenido"]
    if pd.isnull(texto):
        continue
    chunks = splitter.split_text(texto)
    for i, chunk in enumerate(chunks):
        chunked_docs.append(Document(
            page_content=chunk,
            metadata={
            "titulo": row["titulo"],
            "url": row["url"],
            "precio": row.get("Precios", ""),
            "fecha": row.get("fechas", ""),
            "contexto_fecha":row.get("fechas_contexto",""),
            "edad": row.get("edad", ""),
        }
    )
        )

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Vectorstore
vectorstore = FAISS.from_documents(chunked_docs, embedding_model)

In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

from langchain_community.llms import Ollama

# 1. Configura tu modelo local con Ollama
llm = Ollama(model="mistral")  # Asegúrate de ejecutar previamente: ollama serve mistral

# 2. Define el estado
class AgentState(TypedDict):
    query: str
    documents: List[Document]
    generation: str

# 3. Recuperación con MMR (ajustado para diversidad)
def retrieve(state):
    query = state["query"]
    docs = vectorstore.max_marginal_relevance_search(query, k=10, fetch_k=50)
    return {"documents": docs}

# 4. Generación usando contexto real
def generate(state):
    docs = state["documents"]
    query = state["query"]
    context = "\n\n".join(doc.page_content for doc in docs)

    prompt = f"""
Contesta exclusivamente a la pregunta usando la información proporcionada. 
No inventes detalles ni enlaces si no aparecen en el contexto. 
Si no hay suficiente información, indícalo claramente.

Contexto:
{context}

Pregunta: {query}

Respuesta informativa y honesta:
"""

    respuesta = llm.invoke(prompt)

    # Añade enlaces de los artículos utilizados
    enlaces = {
        doc.metadata.get("url"): doc.metadata.get("titulo", "Sin título")
        for doc in docs if doc.metadata.get("url")
    }
    links_texto = "\n\nEnlaces relacionados:\n" + "\n".join(f"{titulo} → {url}" for url, titulo in enlaces.items())

    return {
        "generation": f"{respuesta.strip()}{links_texto}"
    }

# 5. Arma el grafo del agente
graph = StateGraph(AgentState)
graph.add_node("retrieve", retrieve)
graph.add_node("generate", generate)
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "generate")
graph.add_edge("generate", END)

rag_agent = graph.compile()


C:\Users\Abdon\AppData\Local\Temp\ipykernel_22556\872245027.py:10: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")  # Asegúrate de ejecutar previamente: ollama serve mistral


In [8]:
# 6. Ejecuta una prueba
respuesta = rag_agent.invoke({"query": "¿Qué actividades hay este fin de semana de pago?"})
print(respuesta["generation"])

Este fin de semana en Madrid, hay varias actividades que podrías encontrar. Una de ellas es la carrera solidaria y educativa del Museo de la Felicidad, dedicada a la serie Friends, que se realiza en Espacio Ibercaja Delicias. La entrada varía dependiendo del tipo de paquete elegido y su precio va desde los 20 hasta los 150 euros.
   Además, en el Museo de la Felicidad, también puedes disfrutar de una visita diferente que combina diversión, aprendizaje y bienestar, perfecta para personas de todas las edades.
   Por otro lado, la Bolsa Mágica es un espacio familiar de juego libre especializado en la etapa 0-3 años donde los niños pueden explorar, experimentar y socializar en un entorno seguro y preparado. Allí hay actividades sensoriales, música, cuentos y talleres guiados.
   Es recomendable siempre verificar la agenda de eventos en Madrid y preguntar por posibles ofertas y promociones que se pueden encontrar en estos lugares.

Enlaces relacionados:
Los mejores restaurantes temáticos en